# **All Trump's Twitter Insults**

* **beginner**
* **eda**
* **dataviz**
* **nlp**


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# **Exploratory Data Analysis**

In [ ]:
df = pd.read_csv('/kaggle/input/all-trumps-twitter-insults-20152021/trump_insult_tweets_2014_to_2021.csv')

In [ ]:
df.info()

In [ ]:
df.head(2)

In [ ]:
# dropping Unnamed: 0 column
df = df.drop(df.columns[0], axis=1)
df.head(2)

In [ ]:
# Find NA values
df.isna().sum()

In [ ]:
# fill NA values 
df.target = df.target.fillna('unknown')
df.isna().sum()

In [ ]:
# create column datetime.year value
df['year'] = pd.DatetimeIndex(df['date']).year

In [ ]:
df.head(2)

# **Data Visualization - EDA**

In [ ]:
# Importing pygal and its styles
!pip install pygal -q
import pygal
from pygal.style import Style
from IPython.display import display, HTML

In [ ]:
base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""
def pygalplot(chart):
    rendered_chart = chart.render(is_unicode=True)
    plot_html = base_html.format(rendered_chart=rendered_chart)
    display(HTML(plot_html))

In [ ]:
year = df.groupby('year')['year'].count().sort_values(ascending=False)
line_chart = pygal.Bar()
line_chart.title = 'Insults by year'
for p in list(year.index):
    line_chart.add(str(p), year.loc[p])
pygalplot(line_chart)

In [ ]:
target = df.groupby('target')['target'].count().sort_values(ascending=False)
line_chart = pygal.HorizontalBar()
line_chart.title = 'Insults by target'
for p in range(0, 20):
    line_chart.add(target.index[p], target.values[p])
pygalplot(line_chart)

In [ ]:
insult = df.groupby('insult')['insult'].count().sort_values(ascending=False)
line_chart = pygal.HorizontalBar()
line_chart.title = 'Top Insults'
for p in range(0, 20):
    line_chart.add(insult.index[p], insult.values[p])
pygalplot(line_chart)

In [ ]:
df.insult = df.insult.apply(lambda x: x.lower())
insults = df.insult.unique().tolist()
df['tokenInsult'] = df.insult.apply(lambda x: insults.index(x))
insultsId = df.tokenInsult.unique().tolist()


In [ ]:
topInsults = df[['insult', 'tweet']].groupby(['insult']).count().sort_values(by='tweet', ascending=False)

line_chart = pygal.Bar()
line_chart.title = 'Insults by year'
line_chart.x_labels = map(str, df.year.unique().tolist())
tbyYear = []
for p in topInsults[:20].index.tolist():
    total = []
    for y in df.year.unique().tolist():
      total.append(df.query(f'year=={y} and tokenInsult == {insults.index(p)}')['year'].count())
    tbyYear.append(total)
y=0
for p in topInsults[:20].index.tolist():  
    line_chart.add(str(p).upper(), tbyYear[y])
    y+=1
    

pygalplot(line_chart)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

def myWordCloud(data, title, width = 1000, height = 600,):
  wordcloud = WordCloud(width = width, height = height, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(' '.join(data)) 
  
  # plot the WordCloud image                        
  plt.figure(figsize = (8, 8), facecolor = None) 
  plt.imshow(wordcloud) 
  plt.axis("off") 
  plt.tight_layout(pad = 0) 
  plt.title(title)
  plt.show()

In [ ]:
cv = CountVectorizer()
matriz = cv.fit_transform(df.insult)
contaPalavra = pd.DataFrame(cv.get_feature_names(), columns={'Word'})
contaPalavra['Counter'] = matriz.sum(axis=0).tolist()[0]
contaPalavra = contaPalavra.sort_values('Counter', ascending=False).reset_index(drop=True)

myWordCloud(contaPalavra.Word, f'Top Insults from Donald Trump')

In [ ]:
df.target = df.target.apply(lambda x: x.lower())
targets = df.target.unique().tolist()
df['tokenTarget'] = df.target.apply(lambda x: targets.index(x))
targetsId = df.tokenTarget.unique().tolist()

In [ ]:
topTargets = df[['target', 'tweet']].groupby(['target']).count().sort_values(by='tweet', ascending=False)

line_chart = pygal.Bar()
line_chart.title = 'Target from Trump by year'
line_chart.x_labels = map(str, df.year.unique().tolist())
tbyYear = []
for p in topTargets[:20].index.tolist():
    total = []
    for y in df.year.unique().tolist():
      total.append(df.query(f'year=={y} and tokenTarget == {targets.index(p)}')['year'].count())
    tbyYear.append(total)
y=0
for p in topTargets[:20].index.tolist():  
    line_chart.add(str(p).upper(), tbyYear[y])
    y+=1
    

pygalplot(line_chart)

In [ ]:
cv = CountVectorizer()
matriz = cv.fit_transform(df.target)
contaPalavra = pd.DataFrame(cv.get_feature_names(), columns={'Word'})
contaPalavra['Counter'] = matriz.sum(axis=0).tolist()[0]
contaPalavra = contaPalavra.sort_values('Counter', ascending=False).reset_index(drop=True)

myWordCloud(contaPalavra.Word, f'Top Targets from Donald Trump')

# **Dataviz - NLP (Sentiment)**

In [ ]:
!pip install vaderSentiment -q
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
readSentiment = SentimentIntensityAnalyzer()
def getSentiment(phrase):
  s = readSentiment.polarity_scores(phrase)
  if s['compound'] <= -0.05:
    sentiment = 0
  elif s['compound'] >= 0.05:
    sentiment = 1
  else:
    sentiment = 2
  return sentiment, s

In [ ]:
phrases = ["I love the team and how they played last night 💘" ,
"What a fine day I am having today :-) :-)",
"I am laughing like crazy lol",
"He was not very good at the play",
"To be or not to be",
"He is kinda bored",
]
sentiments = ['Negative', 'Positive', 'Neutral']
for txt in phrases:
  print(sentiments[getSentiment(txt)[0]], ' - ', txt)

In [ ]:
f'Qty rows: {df.shape[0]}'

In [ ]:

df['sentimentVader'] = df.tweet.apply(lambda x: sentiments[getSentiment(x)[0]])
df['sentimentVader'].sample(10)

In [ ]:
sentiment = df.groupby('sentimentVader')['sentimentVader'].count()
sentiment

In [ ]:
# sentiment
pie_chart = pygal.Pie()
pie_chart.title = 'Insult vs Sentiment (by Vader)'
for p in range(0, len(sentiment)):
    pie_chart.add(sentiment.index[p], sentiment.values[p])
pygalplot(pie_chart)